In [1]:
!pip -q install transformers accelerate bitsandbytes sentencepiece huggingface_hub


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 13.7 MB/s eta 0:00:00


In [ ]:
import torch

print("CUDA available:", torch.cuda.is_available())
print("GPU name:", torch.cuda.get_device_name(0))


CUDA available: True
GPU name: Tesla T4


In [2]:
from huggingface_hub import login
login()


In [4]:
from huggingface_hub import HfApi

api = HfApi()
print("Access OK:", api.model_info("meta-llama/Meta-Llama-3-8B-Instruct").modelId)


Access OK: meta-llama/Meta-Llama-3-8B-Instruct


In [5]:
import os
from huggingface_hub import HfFolder

hf_token = HfFolder.get_token()
print("Token loaded:", hf_token is not None)


Token loaded: True


In [6]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from huggingface_hub import HfFolder

hf_token = HfFolder.get_token()

model_id = "meta-llama/Meta-Llama-3-8B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(
    model_id,
    token=hf_token
)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype=torch.float16,
    load_in_4bit=True,
    token=hf_token
)

print("LLaMA 3.1 loaded successfully")


tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


...

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

LLaMA 3.1 loaded successfully


In [7]:
def ask_llama_chat(question):
    messages = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": question}
    ]

    prompt = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    output = model.generate(
        **inputs,
        max_new_tokens=150,
        do_sample=False,
        eos_token_id=tokenizer.eos_token_id
    )

    decoded = tokenizer.decode(output[0], skip_special_tokens=True)

    # 🔴 REMOVE THE PROMPT FROM THE OUTPUT
    answer = decoded.split("assistant")[-1].strip()

    return answer


In [8]:
!pip install pyngrok


In [9]:
from flask import Flask, request, jsonify
from pyngrok import ngrok
import threading

app = Flask(__name__)

@app.route("/generate", methods=["POST"])
def generate():
    try:
        data = request.get_json()

        # ✅ LIRE prompt (PAS question)
        prompt = data.get("prompt", "")

        if not prompt:
            return jsonify({"answer": "I don't know."})

        print("🔹 Prompt length:", len(prompt))

        answer = ask_llama_chat(prompt)

        return jsonify({"answer": answer})

    except Exception as e:
        print("❌ LLM ERROR:", str(e))
        return jsonify({
            "error": "LLM generation failed",
            "details": str(e)
        }), 500


@app.route("/")
def root():
    return {"status": "LLM running"}


def run_flask():
    app.run(host="0.0.0.0", port=5005)

threading.Thread(target=run_flask).start()

ngrok.set_auth_token("37DWIx32GfWMNWVlyovpmj7CkDX_2NvbHSh5XsReyvrXJM4CP")
public_url = ngrok.connect(5005)

print("🚀 LLM API disponible sur :", public_url)


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5005
 * Running on http://172.28.0.12:5005
INFO:werkzeug:Press CTRL+C to quit


🚀 LLM API disponible sur : NgrokTunnel: "https://nonreigning-prettiest-shonta.ngrok-free.dev" -> "http://localhost:5005"


In [10]:
import requests
r = requests.post("https://nonreigning-prettiest-shonta.ngrok-free.dev//generate", json={"prompt":"Say hi"})
print(r.json())


The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


🔹 Prompt length: 6


INFO:werkzeug:127.0.0.1 - - [25/Dec/2025 19:25:44] "POST /generate HTTP/1.1" 200 -


{'answer': "Hi there! It's nice to meet you! I'm here to help with any questions or tasks you might have. What's on your mind today?"}
